# NTRU Encrypt - PKE Scheme

O sistema criptográfico NTRU é um sistema de chave pública baseado em reticulados, tendo sido o primeiro deste tipo. 
A implementação desenvolvida neste trabalho corresponde ao criptosistema de chave pública `ntru-pke` que se baseia
na especificação original do NTRU desenvolvida por *Hoffstein, Pipher* e *Silverman*.


## Sampler

A classe `Sampler` apresentada permite gerar polinómios ternários (**i.e.** com coeficientes -1,0,1) pseudo-aleatórios. Esta classe recorre a uma `seed` que determina a sequência obtida quando os parâmetros de entrada
- `N` : ordem do polinómio
- `d1` : número de coeficientes **1**'s
- `d2` : número de coeficientes **-1**'s
permanecem constantes.

A especificação deste `Sampler`(gerador) para gerar um polinómio binário, presente "**EESS #1: Implementation Aspects of NTRUEncrypt and pqNTRUSign**", apresenta-se de seguida:

1. f = 0
1. t = 0
1. while t < d1: 
    1. k = pop( log(N) + 1 ) bits from seed
    1. i = k % N
    1. if f$_i$ == 0:
        1. f$_i$ = 1
        1. t += 1
1. return f.

onde f corresponde ao polinómio gerado e f$_i$ ao coeficiente de ordem $i$ desse polinómio.
Este algoritmo é implementado pelo método `genBinaryPolynomial`.

O método `genTrinaryPolynomial` gera o polinómio ternário recorrendo ao método `genBinaryPolynomial` para gerar
um polinómio binário e, posteriormente, inserir os coefiecientes **-1**'s.

In [3]:
class Sampler:
    def __init__(self, N, d1, d2, seed):
        self.N = N
        if d1 and d2:
            self.d1 = d1
            self.d2 = d2
        self.seed = seed
        
    def genBinaryPolynomial(self):
        ''' Generate a binary polynomial with d1 1's
        '''
        f = [0]*self.N
        t = 0
        while t < self.d1:
            bit_count = ceil(log(self.N)) + 1
            k = self.seed%(2**bit_count)
            self.seed >>= bit_count
            i = k % self.N
            if f[i] == 0:
                f[i] = 1
                t += 1
        return f
    
    def genTrinaryPolynomial(self):
        ''' Generate a trinary polynomial with d1 1's and d2 2's
        '''
        f = self.genBinaryPolynomial()
        t = 0
        while t < self.d2:
            bit_count = ceil(log(self.N)) + 1
            k = self.seed%(2**bit_count)
            self.seed >>= bit_count
            i = k % self.N
            if f[i] == 0:
                f[i] = -1
                t += 1
        return f 

## Algoritmo de Geração de chaves

**INPUT**: Um conjunto de parâmetros $Param = \{N, p, q, d\}$ e uma $seed$.

1. Instanciar $Sampler$ com $\tau(d + 1, d)$ e com a $seed$;
2. $f ← Sampler$
3. se $f$ não é invertível $mod q$ então retornar ao passo 2
4. $g ← Sampler$
5. $h = g/(pf + 1) mod q$

**OUTPUT**: Chave pública $h$ e chave secreta $(pf, g)$

## Algoritmo de Cifragem

**INPUT**: Chave pública h, mensagem msg de comprimento mlen, um conjunto de parâmetros $Param$ e a $seed$.
1. $m = Pad(msg, seed)$
1. $rseed = Hash(m|h)$
1. Instanciar $Sampler$ com $T$ e com $rseed$;
1. $r ← Sampler$
1. $t = r ∗ h$
1. $tseed = Hash(t)$
1. Instanciar $Sampler$ com $T$ e com $tseed$;
1. $m_{mask} ← Sampler$
1. $m' = m − m_{mask}(mod p)$
1. $c = t + m$

**OUTPUT**: Criptograma **c**

Este algoritmo usa um **método de *padding*** que lida com casos em que a mensagem apresente entropia insuficiente. Supondo que o tamanho da mensagem é válido e menor que $(N - 173)$ *bits*, o algoritmo *padding* funciona da seguinte maneira:

1. Converter $msg$ numa *string* de *bits* que constituem os coeficientes do polinómio $m (x)$

2. Os últimos $167$ coeficientes de $m (x)$ são escolhidos aleatoriamente de $\{−1, 0, 1\}$ (com um *input* *seed* )

3. O comprimento da $msg$ é convertido numa *string* binária de 8 *bits* e forma os últimos $173$ a $168$ coeficientes de $m (x)$.

## Algorithm de Decifragem

**INPUT**: Chave secreta $f$, chave pública $h$, criptograma $c$, e o conjunto de parâmetros $Param$.
1. $m' = f ∗ c (mod p)$
1. $t = c − m$
1. $tseed = Hash(t)$
1. Instanciar $Sampler$ com $T$ e com $tseed$;
1. $m_{mask} = Sampler$
1. $m = m' + m_{mask} (mod p)$
1. $rseed = Hash(m|h)$
1. Instanciar $Sampler$ com $T$ e com $rseed$;
1. $r ← Sampler$
1. $msg, mlen = Extract(m)$
1. se $p · r ∗ h = t$ então:
    1. $result = msg, mlen$
1. senão:
    1. $result = ⊥ $

**OUTPUT**: $result$

No algoritmo acima a operação $Extrair ()$ corresponde ao inverso de $Pad ()$. Emite uma mensagem $m$ e seu comprimento $mlen$.

In [4]:
from hashlib import sha512

class NTRU(object):
    def __init__(self, manual, d, N=None, p=None, q=None):
        if manual:
            self.d = d
            self.N = N
            self.p = p
            self.q = q
        else:
            self.d = d
            self.N = next_prime(1 << self.d)
            self.p = 3
            self.q = next_prime(self.p*self.N)
        Z.<x>  = ZZ[]
        Q.<x>  = PolynomialRing(GF(self.q),name='x').quotient(x^self.N-1)
        self.Q = Q
        self.keygen()
        
    def keygen(self):
        # Instantiate Sampler with T (d + 1, d) and seed;
        seed =  int(sha512(bytes("12122121")).hexdigest(), 16)
        sampler = Sampler(self.N, self.d, self.d+1, seed)
        F = self.Q(sampler.genTrinaryPolynomial())
        f = 1 + self.p*F
        while not f.is_unit():
            F = self.Q(sampler.genTrinaryPolynomial())
            f = 1 + self.p*F
        g = self.p*self.Q(sampler.genTrinaryPolynomial())
        h = self.rnd_modq(g/f)
        self.skey = (f,g)
        self.h = h

    def encrypt(self, msg, seed):
        padded_msg = self.pad_msg(msg, seed)

        rseed = int(sha512(bytes(padded_msg + self.h)).hexdigest(), 16)
        sampler = Sampler(self.N, self.d, self.d+1, rseed)
        r = self.Q(sampler.genTrinaryPolynomial())
        t = r*self.Q(self.h)

        tseed = int(sha512(bytes(t)).hexdigest(), 16)
        sampler = Sampler(self.N, self.d, self.d+1, tseed)
        msg_mask = self.Q(sampler.genTrinaryPolynomial())
        
        masked_msg = self.rnd_modp(self.rnd_modq(self.Q(padded_msg) - msg_mask))
        crypt = t + self.Q(masked_msg)
        return crypt.list()

    def decrypt(self, crypt):
        crypt = self.Q(crypt)
        masked_msg = self.rnd_modp(self.rnd_modq(self.skey[0] * crypt))
        t = crypt - self.Q(masked_msg)
        
        tseed = int(sha512(bytes(t)).hexdigest(), 16)
        sampler = Sampler(self.N, self.d, self.d+1, tseed)
        msg_mask = self.Q(sampler.genTrinaryPolynomial())
        padded_msg = self.rnd_modp(self.rnd_modq(self.Q(masked_msg) + msg_mask))

        rseed = int(sha512(bytes(padded_msg + self.h)).hexdigest(), 16)
        sampler = Sampler(self.N, self.d, self.d+1, rseed)
        r = self.Q(sampler.genTrinaryPolynomial())
        
        if r*self.Q(self.h) == t:
            msg, mlen = self.extract_msg(padded_msg)
            result = msg[:mlen]
        else:
            result = "Error"
        return result
    
    def rnd_modq(self, l):
        '''Round f mod q
        '''
        qq = (self.q-1)//2
        ll = map(lift,l.list())
        return [n if n <= qq else n - self.q  for n in ll]

    def rnd_modp(self, l):
        '''Round l mod p
        '''
        pp = (self.p-1)//2
        rr = lambda x: x if x <= pp else x - self.p
        return [rr(n%self.p) if n>=0 else -rr((-n)%self.p) for n in l]

    def pad_msg(self, msg, seed):
        ''' Pad message according to NTRU spec
        '''
        mlen = len(msg)
        if mlen < (self.N - 173):
            fill_len = self.N-173-mlen+1
            sampler = Sampler(167, self.d, self.d+1, seed)
            rand_pad = sampler.genTrinaryPolynomial()
            padded_msg = copy(msg)
            padded_msg.extend([0] * fill_len)
            padded_msg.extend(rand_pad)
            for i in range(5):
                padded_msg.append(mlen%2)
                mlen >>= 1
        return padded_msg
            
    def extract_msg(self, padded_msg):
        ''' Reverse padded message
        '''
        msg_len = 0
        for i in range(1,6):
            msg_len = msg_len*2+padded_msg[-i]
        return padded_msg, msg_len 

In [5]:
K = NTRU(False, 10)
#K = NTRU(True, 10, N=443, q=2048, p=3) #NTRU-443
msg = [choice([-1,0,1]) for i in range(10)]
seed =  int(sha512(bytes("12122121")).hexdigest(), 16) #seed
e = K.encrypt(msg, seed)
d = K.decrypt(e)
print(d==msg)

True
